# Purpose of this notebook

Do something useful with the  [relational data model](https://opendata.tweedekamer.nl/documentatie/informatiemodel) that the tweede kamer provides -- see also 
[extras_datacollect_tweede_kamer_dossiers](extras_datacollect_tweede_kamer_dossiers.ipynb).


Our quest is "make a list of Who is part of what party.

This involves four different resources: Persoon, FractieZetelPersoon, FractieZetel, Fractie

You can see FractieZetelPersoon is a many-to-many joiner table
- a reference to a FractieZetel, which itself just references a Fractie
- a reference to a and Persoon

Persoon and Fractie have no references so are just details to each item.

The below code does relational joins for party membership, 
then flattens the results by adding membership to the persoon details.

In [1]:
import json, pprint

import wetsuite.datacollect.tweedekamer_nl   # contains some basic code dealing with this API
import wetsuite.helpers.etree

In [2]:
# Fetch all resources of the mentioned soort.
# the combination should take a minute or two.
data_Persoon            = wetsuite.datacollect.tweedekamer_nl.merge_etrees( wetsuite.datacollect.tweedekamer_nl.fetch_all( 'Persoon' ) )
data_Fractie            = wetsuite.datacollect.tweedekamer_nl.merge_etrees( wetsuite.datacollect.tweedekamer_nl.fetch_all( 'Fractie' ) )
data_FractieZetel       = wetsuite.datacollect.tweedekamer_nl.merge_etrees( wetsuite.datacollect.tweedekamer_nl.fetch_all( 'FractieZetel' ) )
data_FractieZetelPesoon = wetsuite.datacollect.tweedekamer_nl.merge_etrees( wetsuite.datacollect.tweedekamer_nl.fetch_all( 'FractieZetelPersoon' ) )

In [7]:
# go through entries or each of those, split into individual items
#   mostly to make it slightly easier to later fetch fetch them, and their values
id_thing = {}   #               guid -> detailsdict
by_type  = {}   #     soort/category -> list of all such detailsdicts

for etree in (data_Persoon, data_Fractie, data_FractieZetel, data_FractieZetelPesoon):
    for edict in wetsuite.datacollect.tweedekamer_nl.entries( etree ):
        id_thing[ edict['id'] ] = edict

        category = edict['category']
        if category not in by_type:
            by_type[category] = []
        by_type[ category ].append( edict )

In [11]:
# This is basically a manual JOIN

persoon_combined = {}  # which is mostly still the persoon dicts, except we added a key that is the membership

for fzp_dict in by_type['fractieZetelPersoon']:
    if 'fractieZetel' not in fzp_dict:
        print("  FractieZetelPersoon item without FractieZetel - huh? %r"%fzp_dict)
        continue

    _, fractiezetel_id = fzp_dict['fractieZetel']
    _, persoon_id      = fzp_dict['persoon']

    fractiezetel_dict = id_thing[ fractiezetel_id   ]
    persoon_dict      = id_thing[ persoon_id ]

    _, fractie_id = fractiezetel_dict['fractie']
    frac_dict = id_thing[ fractie_id ]

    if persoon_id not in persoon_combined:
        persoon_combined[ persoon_id ] = persoon_dict
        persoon_combined[ persoon_id ]['fractie_membership'] = []
        
    persoon_combined[ persoon_id ]['fractie_membership'].append( 
        {
            'fractie_id':frac_dict['id'], 
            'fractie_afkorting':frac_dict['afkorting'], 
            # maybe more or all of frac_dict?
            'functie':fzp_dict['functie'], 
            'van':fzp_dict['van'], 
            'totEnMet':fzp_dict['totEnMet'],
        } 
    )


COMBINING
  FractieZetelPersoon item without FractieZetel - huh? {'updated': '2023-08-29T14:00:09Z', 'category': 'fractieZetelPersoon', 'id': 'd73d7f69-1235-4746-aa94-84b593909bfc'}


In [13]:

print("WRITING fractie_membership.json")
with open('fractie_membership.json','wb') as jsonfile:
    jsonfile.write( json.dumps( persoon_combined ).encode('ascii') )

print("WRITING fracties.json")
fracs = {} # id -> details, for consistency with the above and easier lookup
for fracs_dict in by_type['fractie']:
    fracs[ fracs_dict['id'] ] = fracs_dict

with open('fracties.json','wb') as jsonfile:
    jsonfile.write( json.dumps( fracs ).encode('ascii') )


WRITING fractie_membership.json
WRITING fracties.json
